### Инструкция
Используя исходные или очищенные данные, сформируйте предсказание класса объявления из множества exposition_test.tsv.gz

Обязательно нужно использовать одну или несколько моделей кластеризации. Дополнительно можно использовать решающие деревья, CatBoost, LightGBM и XGBoost.

Подсказка: для использования day_mean в классификации/кластеризации потребуется его сформировать для тестовых данных. Это можно сделать либо при помощи других моделей (два этапа классификации), либо построив линейную модель прогноза day_mean от count_day.

Данные:
* https://video.ittensive.com/machine-learning/hacktherealty/E/exposition_train.tsv.gz
* https://video.ittensive.com/machine-learning/hacktherealty/E/exposition_test.tsv.gz
* https://video.ittensive.com/machine-learning/hacktherealty/data/metro.utf8.json
* https://video.ittensive.com/machine-learning/hacktherealty/E/exposition_sample_submisson.tsv

## Часть 1 

Выполнение задания разбито на 2 части.

В первой части производим разведывательный анализ данных, обогащаем дополнительными данными и выделяем из всего многообразия факторов самые значимые.

Сохраняем получившийся набор данных с самыми значимыми наборами как некоторый базовый набор, который будем использовать в части 2. 


### Загрузка библиотек

In [1]:
import numpy as np
import pandas as pd
from transliterate import translit  # для транслитерации
from tqdm import notebook  # прогресс бар пандас
from haversine import haversine  # для добавления данных о метро по координатам
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
from sklearn_som.som import SOM


In [2]:
# вспомогательная функция позволяющая обогатить данные 
def data_preproccesing (data):
# add total items per day
    data_day_count = data.groupby("day").count()["build_year"]
    data["day_count"] = data["day"].apply(lambda x:data_day_count.loc[x])
# approximate values (clean-up)
    data.loc[data.build_year == 0, 'build_year'] = np.NaN
    data['build_year'] = data['build_year'].fillna((data.groupby(['building_series_id'])['build_year'].transform('median')))
    data.loc[data['build_year'].isna(), 'build_year'] = data['build_year'].mean()
    data['build_year'] = data['build_year'].astype(np.uint16)
    if 'has_elevator' in data.columns:
# elevator for 6+ floors
        data.loc[(data.has_elevator==0) & (data.floor>5), 'has_elevator'] = 1
# fix living area
    data.loc[data.living_area == 0, 'living_area'] = np.NaN
    data['living_area'] = data['living_area'].fillna((data.groupby(['rooms'])['living_area'].transform('median')))
# fix price
    data.loc[data.price<100, 'price'] *= 1000  # цена в тыс.
    data.loc[data.price<1000, 'price'] *= 60  # цена в долларах
    if 'floors_total' in data.columns:
# fix celing height  # правим высоту потолков, если данных по высоте серии дома, берем среднее
        data.loc[(data.ceiling_height<2) | (data.ceiling_height>5), 'ceiling_height'] = np.NaN
        data['ceiling_height'] = data['ceiling_height'].fillna(data.groupby(['building_series_id'])['ceiling_height'].transform('median'))
        data.loc[data['ceiling_height'].isna(), 'ceiling_height'] = data['ceiling_height'].mean()
# enrich data, % floor  # этажи как процент этажности
        data['floor'] = data['floor'] / data["floors_total"]
# locality, village/region/moscow/metro  # обогащение по типу города, нас пункту и т.п.
    if 'locality_name' in data.columns:
        data['loctype_village'] = (data['locality_name'].str.match(pat = 'городок|деревня|ДНП|поселок|посёлок|село|СНТ|товарищество|хутор')).astype(np.uint8)
        data['loctype_moscow'] = (data.locality_name == 'Москва').astype(np.uint8)
        data['loctype_region'] = ((data.loctype_village == 0) & (data.loctype_moscow == 0)).astype(np.uint8)
    if "site_id" in data.columns:  # удаляем часть строковых переменных (которые не имеет  смысла переводить в единичные вектора, они не дадут ни какого полезного результата)
        data = data.drop(['site_id', 'main_image', 'area', 'building_id', 'unified_address'], axis=1)
    if 'target_string' in data.columns:
        data = data.drop(['target_string'], axis=1)  # строковое описание таргета
# processing date
    if 'day' in data.columns:  # обогащение по времени 
        data['day'] = pd.to_datetime(data['day'])   # Series.dt.isocalendar().week
        data['year'] = data['day'].dt.year
        data['month'] = data['day'].dt.month
        data['week'] = data['day'].dt.isocalendar().week  #.dt.week
        data['dow'] = data['day'].dt.dayofweek
        data['dom'] = data['day'].dt.day
        data['doy'] = data['day'].dt.dayofyear
        data = data.drop(["day"], axis=1)
# adding holydays, 1-7 Jan, 8 Mar, 1 May, 9 May, 12 Jun, 4 Nov
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2017/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2018/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2019/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2020/
        # добавление выходных
        data['is_holyday'] = ((data['year'] == 2017 &
                                (((data['dom'] > 0) & (data['dom'] < 8) & data['month'] == 1) | 
                                (((data['dom'] == 23) | data['dom'] == 24)) & (data['month'] == 2)) |
                                ((data['dom'] == 8) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 8) | (data['dom'] == 9)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 6) & (data['month'] == 11))) |
                              ((data['year'] == 2018) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 23) & (data['month'] == 2)) |
                                (((data['dom'] == 8) | (data['dom'] == 9)) & (data['month'] == 3)) |
                                ((data['dom'] == 30) & (data['month'] == 4)) |
                                (((data['dom'] == 1) | (data['dom'] == 2) | (data['dom'] == 9)) & data['month'] == 5) |
                                (((data['dom'] == 11) | (data['dom'] == 12)) & (data['month'] == 6)) |
                                ((data['dom'] == 5) & (data['month'] == 11)) |
                                ((data['dom'] == 31) & (data['month'] == 12)))) |
                              ((data['year'] == 2019) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 8) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 2) | (data['dom'] == 3) | (data['dom'] == 9) | (data['dom'] == 10)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 4) & (data['month'] == 11)))) |
                              ((data['year'] == 2020) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 24) & (data['month'] == 2)) |
                                ((data['dom'] == 9) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 4) | (data['dom'] == 5) | (data['dom'] == 11)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 4) & (data['month'] == 11))))).astype(np.uint8)
# one-hot vectors перевод данных в единичные вектора
    if 'year' in data.columns:
        for label in ['year', 'month', 'week', 'dow', 'doy', 'dom', 'renovation',
                      'balcony', 'building_type', 'parking', 'floors_total', 'locality_name']:
            for l in data[label].unique():  # транслитим русские названия 
                data[label + "_" + translit(str(l), "ru", reversed=True)] = (data[label] == l).astype(np.uint8)
# boolean -> int  
    if 'studio' in data.columns:
        for label in ['studio', 'has_elevator', 'expect_demolition', 'is_apartment']:
            data[label] = data[label].astype(np.uint8)
# index (remove id from columns) 
    if 'id' in data.columns:
        data = data.set_index(['id'])
    return data

In [3]:
data_train = pd.read_csv("exposition_train.tsv.gz", sep='\t')  # загрузили данные в формате tsv с разделителем '\t' 

In [4]:
data_train.head()

,building_series_id,site_id,target,parking,target_string,build_year,expect_demolition,main_image,latitude,total_area,...,area,kitchen_area,day,longitude,price,flats_count,building_type,balcony,locality_name,renovation
0,1564812,0,1,OPEN,LESS_7,2005,False,//avatars.mds.yandex.net/get-realty/903734/add...,55.645313,105.000000,...,105.000000,15.0,2018-07-15,37.657490,95000,407,MONOLIT,BALCONY,Москва,EURO
1,1564812,0,2,CLOSED,7_14,2010,False,//avatars.mds.yandex.net/get-realty/1702013/ad...,55.537102,40.000000,...,40.000000,10.0,2019-01-18,37.155632,25000,40,MONOLIT,UNKNOWN,посёлок Первомайское,COSMETIC_DONE
2,663302,0,2,OPEN,7_14,1995,False,//avatars.mds.yandex.net/get-realty/924080/add...,55.662956,37.599998,...,37.599998,0.0,2018-04-24,37.555466,26000,472,PANEL,LOGGIA,Москва,GOOD
3,1564812,0,2,OPEN,7_14,2018,False,//avatars.mds.yandex.net/get-realty/1521999/ad...,55.669151,80.000000,...,80.000000,20.0,2019-02-19,37.285000,35000,156,PANEL,UNKNOWN,Одинцово,GOOD
4,1564812,0,3,UNKNOWN,14_30,2004,False,//avatars.mds.yandex.net/get-realty/50286/f5c8...,55.828518,100.000000,...,100.000000,0.0,2017-08-08,37.361897,80000,31,MONOLIT,UNKNOWN,Москва,EURO


### ETL

* Нужно обогатить данные, по датам (по дням, месяцам, годам), расстоянию до метро, населенном пункту
* Нужно удалить колонки которые не несут полезных данных необходимых для кластеризации и классификации. (Сформировав новый набор данных из имеющихся)


In [ ]:
# воспользуемся функцией data_preproccesing (функция частично авторская от Ittensive, возможно при копировании нужно спросить разрешение)
data_train = data_preproccesing(data_train)
data_train.head()

Добавим среднее значение предсказанной переменной от количества объявлений по дню (day_mean)

In [ ]:
train_data_day_count = data_train.groupby("doy").count()["target"]
train_data_day_mean = data_train.groupby("doy").mean()["target"]

In [7]:
data_train["day_count"] = data_train["doy"].apply(lambda x:train_data_day_count.loc[x])
data_train["day_mean"] = data_train["doy"].apply(lambda x:train_data_day_mean.loc[x])

In [8]:
data_train.head()

,building_series_id,target,parking,build_year,expect_demolition,latitude,total_area,ceiling_height,rooms,floors_total,...,locality_name_sadovoe tovarischestvo Krasnogorskij Sadovod-1,locality_name_sadovoe tovarischestvo Krasnogorskij Sadovod,locality_name_rabochij poselok Proletarskij,locality_name_poselok Novyj gorodok,locality_name_selo Sin'kovo,locality_name_derevnja Lytkino,locality_name_selo Mamontovo,locality_name_derevnja Gorodische,locality_name_poselok Chastsy-1,day_mean
id,,,,,,,,,,,,,,,,,,,,,
5677548107212057955,1564812,1,OPEN,2005,0,55.645313,105.000000,3.00,3,20,...,0,0,0,0,0,0,0,0,0,2.660673
155646401125694364,1564812,2,CLOSED,2010,0,55.537102,40.000000,3.00,1,3,...,0,0,0,0,0,0,0,0,0,2.871636
9186198458182518100,663302,2,OPEN,1995,0,55.662956,37.599998,2.64,0,17,...,0,0,0,0,0,0,0,0,0,3.047502
10844743366553352344,1564812,2,OPEN,2018,0,55.669151,80.000000,3.00,3,27,...,0,0,0,0,0,0,0,0,0,3.001757
3712912186792420056,1564812,3,UNKNOWN,2004,0,55.828518,100.000000,3.00,3,4,...,0,0,0,0,0,0,0,0,0,2.725796


#### Обогащение данных 
Добавление данных о метро (расстояние до станции метро) и средней медианной цены сдачи квартиры, что позволит еще больше повысить точность предсказаний.

Вспомогательные функции для добавления данных о метро (добавление ГЕО данных о метро)

In [9]:
def nearest_metro(house):  # функция поиска станции метро.
    min_dist = 100
    near_metro = ''
    for i in range(len(metro['metro_station'].values)):
        station = (metro['metro_latitude'][i], metro['metro_longitude'][i]) # перебираем все значения по станциям метро
        dist = haversine(house, station)
        if dist < min_dist: # сравниваем координаты дома находя мин расстояние 
            min_dist = dist
            near_metro = metro['metro_station'][i]  
    return [min_dist, near_metro] # возвращаем мин. расстояние и станцию метро

def calculate_nearest_metro(data):  # функция перебора всех домов
    metro_distances = []
    lat = data['latitude'].values  # извлекаем широту и долготу 
    lon = data['longitude'].values
    msk = data['loctype_moscow'].values  # ограничение поиска только для Москвы 
    for i in notebook.tqdm(range(len(lat))):
        if msk[i] == 1:  # если дом в Москве, запуск nearest_metro
            house = (lat[i], lon[i])
            metro_distances.append(nearest_metro(house))
        else:
            metro_distances.append([0, ""])  # если нет просто присваиваем 0
    return np.stack(metro_distances, axis=1)  # возвращаем значения 

def enrich_metro (data, metro_data):  # функция обогащения данными о расстоянии до метро
    data['metro_distance'] = (metro_data[0]).astype(np.float64)
    data['metro_station'] = metro_data[1]
# fill mean values for non-Moscow localities (metro distance is ~ incorrect)
    m = data[data["loctype_moscow"] == 1]["metro_distance"].mean()
    data.loc[data["loctype_moscow"] == 0, "metro_distance"] = m
# one-hot vector for metro station
    for l in data['metro_station'].unique():
        data['metro_station_' + translit(str(l), "ru", reversed=True)] = (data['metro_station'] == l).astype(np.uint8)
    return data

In [10]:
# читаем json с данными о метро
metro = pd.read_json("https://video.ittensive.com/machine-learning/hacktherealty/data/metro.utf8.json")
metro = metro[['NameOfStation', 'Longitude_WGS84', 'Latitude_WGS84']]  # разбираем его на название и координаты 
metro = metro.reset_index().drop('index', axis=1)
metro = metro.rename({'NameOfStation': 'metro_station',
                      'Longitude_WGS84': 'metro_longitude',
                      'Latitude_WGS84': 'metro_latitude'}, axis=1)
metro = metro.drop_duplicates(subset=["metro_station"], keep="first")  # удаляем дубликаты (по входам в метро, если их несколько)
metro = metro.set_index("metro_station").reset_index()
print (metro.head())

              metro_station  metro_longitude  metro_latitude
0               Китай-город        37.631677       55.757315
1                 Калужская        37.539238       55.655386
2             Братиславская        37.752643       55.660114
3  Бульвар адмирала Ушакова        37.541645       55.545011
4  Бульвар Дмитрия Донского        37.576311       55.570289


Добавляем данные по метро

In [11]:
metro_train = calculate_nearest_metro(data_train)

  0%|          | 0/356500 [00:00<?, ?it/s]

In [ ]:
train_data = enrich_metro(data_train, metro_train)

Добавляем данные средней цены квартиры по метро и по городу

In [ ]:
price_data = pd.DataFrame(train_data[["locality_name", "price", "metro_station"]])
# медианная цена по группам город (locality_name) и метро (metro_station)
price_groups = {"locality_name": {
    "median": price_data.groupby(["locality_name"]).median()["price"]
}, "metro_station": {
    "median": price_data.groupby(["metro_station"]).median()["price"]
}}

In [14]:
# функция возвращает отношение текущей цены к медианной цене по району (метро) или по городу
def calc_price (data, group="", label=""):
    if data[group] in price_groups[group][label]:
        return data["price"] / price_groups[group][label][data[group]]
    else:
        return 1

In [15]:
# Добавляем информацию о средней по району / по метро в данные
for group in price_groups:
    print ("Processing:", group, end=" ")
    for label in price_groups[group]:
        print (label, end=" ")
        train_data["price_" + group + "_" + label] = train_data.apply(calc_price, axis=1,
                                                                      group=group, label=label)
    print ("")

Processing: locality_name median 
Processing: metro_station median 


In [16]:
# Сохраним очищенные и обогащенные данные.
data_train.to_csv('exposition_train.csv.gz', index=False) 

In [17]:
data_train.head()

,building_series_id,target,parking,build_year,expect_demolition,latitude,total_area,ceiling_height,rooms,floors_total,...,metro_station_Kuznetskij most,metro_station_Aleksandrovskij sad,metro_station_Borovitskaja,metro_station_Lubjanka,metro_station_Teletsentr,metro_station_Ploschad' Revoljutsii,metro_station_Rumjantsevo,metro_station_Partizanskaja,price_locality_name_median,price_metro_station_median
id,,,,,,,,,,,,,,,,,,,,,
5677548107212057955,1564812,1,OPEN,2005,0,55.645313,105.000000,3.00,3,20,...,0,0,0,0,0,0,0,0,2.261905,2.500000
155646401125694364,1564812,2,CLOSED,2010,0,55.537102,40.000000,3.00,1,3,...,0,0,0,0,0,0,0,0,1.000000,1.086957
9186198458182518100,663302,2,OPEN,1995,0,55.662956,37.599998,2.64,0,17,...,0,0,0,0,0,0,0,0,0.619048,0.553191
10844743366553352344,1564812,2,OPEN,2018,0,55.669151,80.000000,3.00,3,27,...,0,0,0,0,0,0,0,0,1.250000,1.521739
3712912186792420056,1564812,3,UNKNOWN,2004,0,55.828518,100.000000,3.00,3,4,...,0,0,0,0,0,0,0,0,1.904762,2.000000


__Выделение значимых факторов__

Из всего многообразия факторов (атрибутов объявления) объявления необходимо выделить самые значимые для успешного применения моделей машинного обучения.

Воспользуемся линейной регрессией и корреляцией, чтобы оставить некоторый топ факторов (отсечем по уровню корреляции).

Удалим ряд текстовых атрибутов, которые уже разложены в единичные вектора.

In [18]:
data_train = data_train.drop(labels=["parking", "building_type", "balcony",
                                     "renovation", "locality_name", "metro_station"], axis=1)

__Тестирования корреляции параметров и выделения ТОПа факторов__

При помощи линейной регрессии выделим наиболее коррелирующие параметры.

In [19]:
best_features = []
y = data_train["target"]
features_to_test = list(data_train.columns)
for feature in data_train.columns:
    if type(data_train[feature].values[0]) is not str and feature.find("target") == -1:
        x_feature = pd.DataFrame(data_train[feature])
        score = LinearRegression().fit(x_feature, y).score(x_feature, y)
# R^2 >= 0.01, r >= 0.1, 10%
        if score > 0.01:  # вывод факторов с корреляцией не менее 10%
            print (feature, score)
            best_features.append(feature)

total_area 0.047077452744266224
ceiling_height 0.01841105581914615
rooms 0.03143849382456698
living_area 0.04014277880539696
price 0.01283942521085768
doy_108 0.010704859943396428
day_mean 0.027320368318368482
price_locality_name_median 0.011666225448715317


Посмотрим список ТОП параметров

In [20]:
print("Наиболее важные параметры:", best_features)

Наиболее важные параметры: ['total_area', 'ceiling_height', 'rooms', 'living_area', 'price', 'doy_108', 'day_mean', 'price_locality_name_median']


In [21]:
# добавим к ТОПу параметров target
best_features.append("target") 

In [22]:
# сохраним наш ТОП набор, как некоторый базовый, для дальнейшего обучения моделей
data_train[best_features].to_csv('exposition_train.basic.csv.gz', index=False)

### На этом этапе часть 1 заканчивается 